# Workflow test for `micaflow`

## Inputs in BIDS

- BIDS directory example: `/data/mica3/BIDS_MICs/rawdata`

- subject for test `sub-HC126`

- session `ses-01`

- T1w input test: `/data/mica3/BIDS_MICs/rawdata/sub-HC126/ses-01/anat/sub-HC126_ses-01_T1w.nii.gz`


## Output structure
where type can be `T1w`, `FLAIR`, or `T2w`.
https://bids-specification.readthedocs.io/en/stable/modality-specific-files/magnetic-resonance-imaging-data.html


BIDS specifications. 
`OUTDIR/sub-SUBJECT/ses-SESSION/anat/sub-SUBJECT_ses-SESSION_TYPE.nii.gz`


## Dependencies
1. freesurfer 7.4.0

2. ANTS maybe yes maybe no

3. python. What packages?

## workflow 
1. Brain mask `mri_synthstrip` Maybe this step can be avoided and we might be able to use the brain segmentation
2. Bias field correction (ANTs) weighted by white matter
3. Normalization
4. Brain segmentation `synth_seg` from freesurfer
5. Registration to MNI


### Intensity Non-uniform correction - N4
```bash
N4BiasFieldCorrection  -d 3 -i "$T1n4" -r -o "$T1n4" -v
```

### Brain mask
```bash
mri_synthstrip -i "$T1nativepro" -o "$T1nativepro_brain" -m "$T1nativepro_mask" --no-csf
```

### Normalization `02_proc_flair`

### Segmentation 'synth_seg'

```bash
# Define variables
out=/host/yeatman/local_raid/rcruces/data/tmp

img_fixed=/data_/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC019/ses-03/anat/sub-PNC019_ses-03_space-nativepro_T1w_nlm.nii.gz

img_moving=/data_/mica3/BIDS_PNI/derivatives/Denoised_T2star/10PNC_nomask/sub-PNC019_ses-03_B1-corrected_T2starmap_nomask_denoised.nii

id=sub-PNC019_ses-03
outname=${out}/${id}_space-nativepro_T2star.nii.gz

threads=15

# Use synthseg to get the labels of the fixed and moving image
mri_synthseg --i ${img_moving} \
             --o ${out}/${id}_moving_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

mri_synthseg --i ${img_fixed} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

```

### Registration to MNI `easy_reg`
```bash
# Calculate the warpfield and affine transformation with easyreg
mri_easyreg --ref ${img_fixed} \
            --flo ${img_moving} \
            --ref_seg ${out}/${id}_fixed_synthseg.nii.gz \
            --flo_seg ${out}/${id}_moving_synthseg.nii.gz \
            --fwd_field ${out}/${id}_from-moving_to-fixed_mode-image_desc-easyreg.nii.gz \
            --threads ${threads} 
```

## Pre-workflow

> The segmentation of the MNI152 only has to be calculated once

```bash
# Output of the segmentation, this and the nifti must be inside the repository!
atlas_dir=/host/yeatman/local_raid/cerys/Downloads/mni_icbm152_nlin_sym_09a

# MNI152 compresed NIFTI
atlas_mni152_nlin_sym=${atlas_dir}/mni_icbm152_t1_tal_nlin_sym_09a.nii.gz

# Number of threads if CPU is used
threads=15

# Segmentation of the MNI152 non linear symetric 1x1Xx1 mm
# Take into account including or not the option --cpu (optional) Enforce running with CPU rather than GPU.
mri_synthseg --i ${atlas_mni152_nlin_sym} \
             --o ${atlas_dir}/mni_icbm152_t1_tal_nlin_sym_09a_synthseg.nii.gz \
             --robust --vol ${atlas_dir}/mni_icbm152_t1_tal_nlin_sym_09a_volumes.csv \
             --qc ${atlas_dir}/mni_icbm152_t1_tal_nlin_sym_09a_qc.csv --threads ${threads} --cpu --parc
```

# Workflow `micaflow`

1. Create a bash script with all the steps that takes an input and generates the output files
2. See if this can be incorporated within nextflow


## Test script
```bash
micaflow_tmp.sh
  	--subject sub-HC062 \
  	--session ses-03 \
  	--out_dir /host/yeatman/local_raid/cerys/micaflow_test \
  	--fs_license /data_/mica1/01_programs/freesurfer-7.3.2/license.txt \
  	--threads 15
```


## Define variables

In [2]:
%%bash
# ------------------------------------------------------------------------------------
# Mandatory inputs
# ID string
subject=sub-HC062
session=ses-03

# Output directory
out_dir=/host/yeatman/local_raid/cerys/micaflow_test

# Freesurfer license
fs_license_file=/data_/mica1/01_programs/freesurfer-7.3.2/license.txt

# Number of threads if CPU is used
threads=15

# ------------------------------------------------------------------------------------
# Variables
# ID string
BIDS_ID=${subject}_${session}

# Output directory specific to subject and session
out=${out_dir}/${subject}/${session}/anat

# Input image
#T1w=/data/mica3/BIDS_MICs/rawdata/${subject}/${session}/anat/${BIDS_ID}_T1w.nii.gz

T1w=/data_/mica3/BIDS_MICs/rawdata/${subject}/${session}/anat/${BIDS_ID}_run-1_T1w.nii.gz
flair=/data/mica3/BIDS_MICs/rawdata/${subject}/${session}/anat/${BIDS_ID}_FLAIR.nii.gz



## Final Output - not used??
T1_preproc=${out}/${BIDS_ID}_preproc_T1w.nii.gz


# If the directory does not exist create it
mkdir -p $out

## 1. Brain segmentation `mri_synthseg`

In [4]:
%%bash
# Segmentation of the T1w image
# Take into account including or not the option --cpu (optional) Enforce running with CPU rather than GPU.
mri_synthseg --i ${T1w} \
             --o ${out}/${BIDS_ID}_desc-synthseg_T1w.nii.gz \
             --robust --vol ${out}/${BIDS_ID}_desc-volumes_T1w.csv \
             --qc ${out}/${BIDS_ID}_desc-qc_T1w.csv --threads ${threads} --cpu --parc

# Segmentation of the FLAIR image
mri_synthseg --i ${flair} \
             --o ${out}/${BIDS_ID}_desc-synthseg_FLAIR.nii.gz \
             --robust --vol ${out}/${BIDS_ID}_desc-volumes_FLAIR.csv \
             --qc ${out}/${BIDS_ID}_desc-qc_FLAIR.csv --threads ${threads} --cpu --parc

mri_synthseg --i --o /_desc-synthseg_T1w.nii.gz --robust --vol /_desc-volumes_T1w.csv --qc /_desc-qc_T1w.csv --threads 10 --cpu


## Note: If the input image is NOT 1x1x1 the labels wil not match

In [ ]:
# synthseg_resampled is only generated if the voxel sizes don't match; in this case, you will need to do an affine registration back to FLAIR native space
    if [ -f "${tmp}/${idBIDS}_flair_synthseg_resampled.nii.gz" ]; then
        # Affine registration from flair_synthseg_resampled to FLAIR native space
        flair_resample2orig="${tmp}/${idBIDS}_flair_synthresample_to_orig_"
        antsRegistrationSyN.sh -d 3 -f "${flairScan}" -m "${flair_resample}" -o "${flair_resample2orig}" -t a -n "$threads" -p d
        # Apply transformation to flair_synthseg
        flair_synthseg_orig="${tmp}/${idBIDS}_flair_synthseg_orig.nii.gz"
        antsApplyTransforms -d 3 -i "${flair_synthseg}" -r "${flairScan}" -t "${flair_resample2orig}"0GenericAffine.mat -o "${flair_synthseg_orig}" -v -u float -n GenericLabel
        # flair_synthseg in original FLAIR space
        flair_synthseg="${flair_synthseg_orig}"
    fi

## 1.a Extract the white matter segmentation

In [5]:
%%bash
# 1.A | WM white matter mask (lh=2, rh=41)
mri_binarize --i ${out}/${BIDS_ID}_desc-synthseg_T1w.nii.gz --match 2 --match 41 \
--o ${out}/sub-HC126_ses-01_desc-wm_T1w.nii.gz

SyntaxError: invalid syntax (<ipython-input-5-3d2147de6cfa>, line 2)

# 2. Run N4 weighted by white matter

In [7]:
%%bash
# N4 multi-thread requires this on the ENV
export ITK_GET_GLOBAL_DEFAULT_NUMBER_OF_THREADS=${threads}

# Bias field correction (ANTS) for T1w
N4BiasFieldCorrection  -d 3 -i "$T1w" -r -o "${out}/${BIDS_ID}_desc-N4_T1w.nii.gz" -v
#-w ${out}/sub-HC126_ses-01_desc-wm_T1w.nii.gz

# Bias field correction for FLAIR
N4BiasFieldCorrection  -d 3 -i "$flair" -r -o "${out}/${BIDS_ID}_desc-N4_FLAIR.nii.gz" -v

N4BiasFieldCorrection -d 3 -i  -r -o /_desc-N4_T1w.nii.gz -v -w /sub-HC126_ses-01_desc-wm_T1w.nii.gz


# 3. Registration to MNI space

In [1]:
%%bash

# Output of the segmentation, this and the nifti must be inside the repository!
atlas_dir=/host/yeatman/local_raid/cerys/Downloads/mni_icbm152_nlin_sym_09a

# MNI152 compresed NIFTI
atlas_mni152=${atlas_dir}/mni_icbm152_t1_tal_nlin_sym_09a.nii.gz
atlas_mni152_seg=${atlas_dir}/mni_icbm152_t1_tal_nlin_sym_09a_synthseg.nii.gz

# Registration to MNI
mri_easyreg --ref ${atlas_mni152} \
            --ref_seg ${atlas_mni152_seg} \
            --flo ${T1w} \
            --flo_seg ${out}/${BIDS_ID}_desc-synthseg_T1w.nii.gz \
            --fwd_field ${out}/${BIDS_ID}_from-T1w_to-MNI151_1mm_desc-easyreg_fwdfield.nii.gz \
            --bak_field ${out}/${BIDS_ID}_from-T1w_to-MNI151_1mm_desc-easyreg_bakfield.nii.gz \
            --threads ${threads}

2024-04-09 10:35:25.937093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 10:36:07.490048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: mri_easyreg [-h] [--ref REF] [--ref_seg REF_SEG] [--flo FLO]
                   [--flo_seg FLO_SEG] [--ref_reg REF_REG] [--flo_reg FLO_REG]
                   [--fwd_field FWD_FIELD] [--bak_field BAK_FIELD]
                   [--affine_only] [--threads THREADS]
mri_easyreg: error: argument --flo: expected one argument


# 4. Apply warp from `easyreg` T1w to MNI space

In [ ]:
%%bash

# Apply the warpfield
mri_easywarp --i ${out}/${BIDS_ID}_desc-N4_T1w.nii.gz \
             --o ${out}/${BIDS_ID}_space-MNI152_1m_T1w.nii.gz \
             --field ${out}/${BIDS_ID}_from-T1w_to-MNI151_1mm_desc-easyreg_fwdfield.nii.gz \
             --threads ${threads} --nearest

# 5. Align T2/FLAIR to T1w

In [ ]:
%%bash
# Old method don't use
bbregister --s bert \ # freesurfer's subject directory?
           --mov ${flair} \
           --init-fsl
           --reg ${out}/${BIDS_ID}_FLAIR_to_T1w.dat \
           --t1 \
           --o ${out}/${BIDS_ID}_FLAIR_registered_to_T1w.nii.gz

In [ ]:
%%bash
           
# Registration to MNI
mri_easyreg --ref ${out}/${BIDS_ID}_desc-N4_T1w.nii.gz \
            --ref_seg ${out}/${BIDS_ID}_desc-synthseg_T1w.nii.gz \
            --flo ${flair} \
            --flo_seg ${out}/${BIDS_ID}_desc-synthseg_FLAIR.nii.gz \
            --fwd_field ${out}/${BIDS_ID}_from-FLAIR_to-T1w_desc-easyreg_fwdfield.nii.gz \
            --bak_field ${out}/${BIDS_ID}_from-FLAIR_to-T1w_desc-easyreg_bakfield.nii.gz \
            --threads ${threads}

# 6. Use ``mri_easywarp`` with the forward field obtained from the T1w-to-MNI registration to warp T2/FLAIR images to MNI space

In [ ]:
# Apply forward transformation to the FLAIR image
mri_easywarp --i ${out}/${BIDS_ID}_desc-N4_FLAIR.nii.gz \
             --o ${out}/${BIDS_ID}_space-MNI152_1mm_FLAIR.nii.gz \
             --field ${out}/${BIDS_ID}_from-T1w_to-MNI151_1mm_desc-easyreg_fwdfield.nii.gz \
             --threads ${threads} --nearest

# Image data normalization 
https://github.com/rcruces/preproc_reports/blob/main/notebooks/flair_mode-normalization.ipynb

In [ ]:
%%bash
# The the mode of the GM, WM and whole brain
function get_mode() {
  # This functions uses mrhistogram to calculate the mode with awk.
  # Using 1000 bins gets the same results as python
  MRI_img=$1
  MRI_mask=$2
  bins=$3
  # Create histogram file
  hist=${tmp}/hist_masked.txt
  mrhistogram -bins "${bins}" -mask "${MRI_mask}" -ignorezero -nthreads "${threads}" "${MRI_img}" "${hist}"
  # get the index with the max frecuency
  max_val=$(awk -F ',' 'NR==3 { m=$3; p=1; for(i=4;i<=NF;i++) { if ($i>m) { m=$i; p=i-2 } } printf "%d ",p }' "${hist}")
  # bash array of each intensity bin
  intensities=($(awk 'NR==2' "${hist}" | tr -s ',' ' '))
  # get the intensity of the maximun frecuency (aka mode)
  mode=${intensities[$((max_val-1))]}
  # remove tmp file
  rm "${hist}"
  # Print the mode
  echo "${mode}" | awk -F"e" 'BEGIN{OFMT="%10.10f"} {print $1 * (10 ^ $2)}'
}

In [ ]:
%%bash

#------------------------------------------------------------------------------#
# 4 | Get the mode for each tissue
mode_gm=$(get_mode "${flair_N4}" "${flair_mask_gm}" 1000)
mode_wm=$(get_mode "${flair_N4}" "${flair_mask_wm}" 1000)
mode_brain=$(get_mode "${flair_N4}" "${flair_mask}" 1000)

Note "mode_gm    :" "${mode_gm}"
Note "mode_wm    :" "${mode_wm}"
Note "mode_brain :" "${mode_brain}"

#------------------------------------------------------------------------------#
# 5 | Normalize intensities by peak of WM (mode).
# This normalization will center the peak of the WM mode intensity at ZERO.
# Mean mode between GM and WM | BG=(GM_mode+WM_mode)/2.0
BG=$(echo "(${mode_gm}+${mode_wm})/2.0" | bc -l)
# mode difference | mode_diff = np.abs(BG - WM_mode)
mode_diff=$(echo "${BG}-${mode_wm}" | bc); mode_diff=$(echo ${mode_diff#-})
# Normalize array | norm_wm = 100.0 * (array - WM_mode)/(mode_diff)
flair_norm="${tmp}/${idBIDS}_flair_norm.nii.gz"
Do_cmd mrcalc "${flair_N4}" "${mode_wm}" -subtract "${mode_diff}" -div 100 -mul "${flair_norm}"

## Check metrics for registration quality
1. Jaccard index https://en.wikipedia.org/wiki/Jaccard_index
2. Dice score

## Try to find something faster than `N4BiasFieldCorrection`
1. SPM (requires MatLab)
2. FSL's FAST
3. `mri_nu_correct.mni` from freesurfer - slow